In [1]:
import torch
import sys
import argparse
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt

from sam_segment import predict_masks_with_sam
from lama_inpaint import inpaint_img_with_lama
from utils import load_img_to_array, save_array_to_img, dilate_mask, \
    show_mask, show_points, get_clicked_point

/raid/coss07/anaconda3/envs/IA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/raid/coss07/anaconda3/envs/IA/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-16 10:21:15.153749: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 10:21:15.153793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 10:21:15.153829: E tensorflow/com

Detectron v2 is not installed


In [2]:

input_img = '../wim_data/train/images/2023-04-21_48.mp4#t=0.jpg'
point_labels = [1]
sam_model_type = "vit_h"
sam_ckpt = './pretrained_models/sam_vit_h_4b8939.pth'

In [3]:
def normalized_coordinate_to_absolute(norm_x, norm_y, image_width=1280, image_height=720):
    abs_x = int(norm_x * image_width)
    abs_y = int(norm_y * image_height)
    return [abs_x, abs_y]

In [4]:
x_center, y_center = 0.3401102687801516, 0.871380974264706
latest_coords = normalized_coordinate_to_absolute(x_center, y_center)#0.3401102687801516, 0.871380974264706
latest_coords

[435, 627]

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
img = load_img_to_array(input_img)

In [6]:
masks, _, _ = predict_masks_with_sam(
        img,
        [latest_coords],
        point_labels,
        model_type=sam_model_type,
        ckpt_p=sam_ckpt,
        device=device,
    )

In [7]:
masks = masks.astype(np.uint8)* 255

In [8]:
mask = masks[2]

In [9]:
lama_config = 'lama/configs/prediction/default.yaml'
lama_ckpt = './pretrained_models/big-lama'
out_dir = './results'

In [11]:
for idx, mask in enumerate(masks):
    mask_p =  f"./results/mask_cust_{idx}.png"
    img_inpainted_p = f"./results/inpainted_{Path(mask_p).name}"
    img_inpainted = inpaint_img_with_lama(
        img, mask, lama_config, lama_ckpt, device=device)
    save_array_to_img(img_inpainted, img_inpainted_p)

In [12]:
img_inpainted

array([[[ 0, 22,  1],
        [ 0, 22,  1],
        [ 0, 22,  1],
        ...,
        [21, 39,  1],
        [21, 37,  0],
        [21, 37,  0]],

       [[ 0, 22,  1],
        [ 0, 22,  1],
        [ 0, 22,  1],
        ...,
        [22, 40,  0],
        [21, 39,  1],
        [21, 39,  1]],

       [[ 0, 22,  1],
        [ 0, 22,  1],
        [ 0, 22,  1],
        ...,
        [26, 44,  4],
        [26, 42,  3],
        [26, 42,  5]],

       ...,

       [[42, 55,  2],
        [42, 55,  2],
        [42, 55,  2],
        ...,
        [ 0,  2,  0],
        [ 0,  2,  0],
        [ 1,  1,  0]],

       [[42, 55,  2],
        [42, 55,  2],
        [42, 55,  2],
        ...,
        [ 4, 11,  4],
        [ 4, 11,  4],
        [ 4, 11,  4]],

       [[42, 55,  2],
        [42, 55,  2],
        [42, 55,  2],
        ...,
        [28, 55, 36],
        [30, 55, 36],
        [28, 55, 36]]], dtype=uint8)